## 前置-元类(MetaClass)
```
class S:
    pass

print(type(S))
```


In [1]:
# 通过type来创建对象
def hello(self):
    print('say hello')
name = 'Student' # 类名
bases = (object, ) # 父类集合
attrs = {'hello': hello} # 类属性和方法

Student = type(name, bases, attrs)
s = Student()
s.hello()

say hello


In [3]:
# 1. metaclass是类的模板，所以必须从`type`类型派生：
class MetaClass(type):
    def __new__(cls, name, bases, attrs):
        attrs['add'] = lambda self, value: self.append(value)
        return type.__new__(cls, name, bases, attrs)

# 2. 定义了metaclass，接下来我们要定义类，并指定用metaclass来创建：
class MyList(list, metaclass=MetaClass):
    pass

L = MyList()
L.add(1)
L

[1]

In [4]:
class Field:
    def __init__(self, name, column_type):
        self.name = name
        self.column_type = column_type
    
    def __str__(self):
        return '<%s:%s>' % (self.__class__.__name__, self.name)

class StringField(Field):
    def __init__(self, name):
        super(StringField, self).__init__(name, 'varchar(100)')
        
class IntegerField(Field):
    def __init__(self, name):
        super(IntegerField, self).__init__(name, 'bigint')

In [8]:
class ModelMetaclass(type):
    def __new__(cls, name, bases, attrs):
        print('Found model: %s' % name)
        if name == 'Model':
            return type.__new__(cls, name, bases, attrs)
        mappings = dict()
        for k, v in attrs.items():
            if isinstance(v, Field):
                print('Found mapping: %s ==> %s' % (k, v))
                mappings[k] = v
        for k in mappings.keys():
            attrs.pop(k)
        attrs['__mappings__'] = mappings    # 保存属性和列的映射关系
        attrs['__table__'] = name           # 假设表名和类名一致
        return type.__new__(cls, name, bases, attrs)
    
class Model(dict, metaclass=ModelMetaclass):
    def __init__(self, **kwargs):
        print("Model __init__ : ", kwargs )
        super(Model, self).__init__(**kwargs)
        
    def __getattr__(self, key):
        print("__getattr__: %s > %s" % (key,self[key]))
        try:
            return self[key]
        except KeyError:
            raise AttributeError(r"'Model' object has no attribute '%s'" % key)
            
    def __setattr__(self, key, value):
        print("__setattr__ : ", key)
        self[key] = value
        
    def save(self):
        fields = []
        params = []
        args = []
        for k, v in self.__mappings__.items():
            fields.append(v.name)  # id,username,email,password 字段名
            params.append('?')     # ????
            args.append(getattr(self, k, None))
        sql = 'insert into %s (%s) values (%s)' % (self.__table__, ','.join(fields), ','.join(params))
        print('SQL: %s' % sql)
        print('ARGS: %s' % str(args))

Found model: Model


In [9]:
class User(Model):
    id = IntegerField('id')
    name = StringField('name')
    email = StringField('email')
    password = StringField('password')

u = User(id=1124, name='Michael', email='test@orm.org', password='pwd')
u.save()

Found model: User
Found mapping: id ==> <IntegerField:id>
Found mapping: name ==> <StringField:name>
Found mapping: email ==> <StringField:email>
Found mapping: password ==> <StringField:password>
Model __init__ :  {'id': 1124, 'name': 'Michael', 'email': 'test@orm.org', 'password': 'pwd'}
__getattr__: id > 1124
__getattr__: name > Michael
__getattr__: email > test@orm.org
__getattr__: password > pwd
SQL: insert into User (id,name,email,password) values (?,?,?,?)
ARGS: [1124, 'Michael', 'test@orm.org', 'pwd']
